##### Import Packages 

In [ ]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import scipy 

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
import warnings
import seaborn as sns
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.offsetbox as offsetbox
import seaborn.objects as so
from seaborn import axes_style
import matplotlib as mpl
import pickle
from matplotlib import cm

import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature

from global_var_and_func import *
from plot_func import *


warnings.filterwarnings("ignore")
#from sklearn_xarray import Stacker, Select

#### Conventions


- `ts`,`eis`,`tadv`,`rh`,`omega`,`ws`: Short-form for all CCFs: sea surface temperature, estimated inversion strength, advected temperature, relative humidmity, subsidence, and surface wind speed.
- `fast`, `slow`: Year 1-20 and Year 21-150 response from abrupt-4xCO2 respectively. 


- `CCF_perglobts`, `dR_CCF_perglobts`: $dCCF/dT_{g}$ or $dR_{CCF}/dT_{g}$ , calculated as the quotient of $dCCF/dt$ or $dR_{CCF}/dt$ and $dT_{g}/dt$.
- `_smth`: smoothed

##### Data Import

Processed data from `CCF_project_data_processed_EXP.ipynb`, with `EXP` for each experiment. See respective notebooks and manuscript for tables and datasets used from CMIP5 and CMIP 6 data. 

`Kernel.pkl` are pickled kernel data from [Scott et al. 2020](: https://doi.org/10.1175/JCLI-D-19-1028.1) and [Myers et al. 2021](https://doi.org/10.1038/s41558-021-01039-, accessed from https://github.com/tamyers87/meteorological_cloud_radiative_kernels?tab=readme-ov-file. . 

In [ ]:
#AMIP data
ts_amip,eis_amip,tadv_amip,rh_amip,omega_amip,ws_amip = get_data('amip')
#historical data
ts_hist,eis_hist,tadv_hist,rh_hist,omega_hist,ws_hist = get_data('obshist')
#abrupt data
ts_abrupt,eis_abrupt,tadv_abrupt,rh_abrupt,omega_abrupt,ws_abrupt = get_data('4xCO2')
ts_fast, ts_slow = extract_abrupt(ts_abrupt)
eis_fast, eis_slow = extract_abrupt(eis_abrupt)
tadv_fast, tadv_slow = extract_abrupt(tadv_abrupt)
rh_fast, rh_slow = extract_abrupt(rh_abrupt)
omega_fast, omega_slow = extract_abrupt(omega_abrupt)
ws_fast, ws_slow = extract_abrupt(ws_abrupt)

#Kernel Data
with open('kernel.pkl', 'rb') as f:
    (sst_kernels, eis_kernels, tadv_kernels, rh_kernels, omega_kernels, ws_kernels,
     sst_obs_kernels, eis_obs_kernels, tadv_obs_kernels, rh_obs_kernels, omega_obs_kernels, ws_obs_kernels) = pickle.load(f)

##### dCCF/dT (d[CCF unit]]/dK)

###### Smoothed with Rolling mean per every 5 years

Ran `perglobts_add_cyclic_point` on account of projection discontinuity with `central_longitude=180`.

In [ ]:
# variables for the above calculations per experiment (AMIP, historical and abrupt4xCO2)
sst_perglobts_smth = ccf_changes_perglobts(ts_amip,ts_amip, ts_hist,ts_hist,\
                                           ts_fast,ts_fast, ts_slow,ts_slow,smth=True) #, sst_perglobts_glob_smth, sst_perglobts_hat_smth, sst_perglobts_hat_zonal_smth, sst_perglobts_hat_glob_smth

eis_perglobts_smth = ccf_changes_perglobts(ts_amip,eis_amip, ts_hist,eis_hist,\
                                           ts_fast,eis_fast, ts_slow,eis_slow,smth=True) # , eis_perglobts_glob_smth, eis_perglobts_hat_smth, eis_perglobts_hat_zonal_smth, eis_perglobts_hat_glob_smth

tadv_perglobts_smth = ccf_changes_perglobts(ts_amip,tadv_amip, ts_hist,tadv_hist,\
                                            ts_fast,tadv_fast, ts_slow,tadv_slow,smth=True) #, tadv_perglobts_glob_smth, tadv_perglobts_hat_smth, tadv_perglobts_hat_zonal_smth, tadv_perglobts_hat_glob_smth

rh_perglobts_smth = ccf_changes_perglobts(ts_amip,rh_amip, ts_hist,rh_hist,\
                                          ts_fast,rh_fast, ts_slow,rh_slow,smth=True) #, rh_perglobts_glob_smth, rh_perglobts_hat_smth, rh_perglobts_hat_zonal_smth, rh_perglobts_hat_glob_smth

omega_perglobts_smth = ccf_changes_perglobts(ts_amip,omega_amip, ts_hist,omega_hist,\
                                             ts_fast,omega_fast, ts_slow,omega_slow,smth=True) #, omega_perglobts_glob_smth, omega_perglobts_hat_smth, omega_perglobts_hat_zonal_smth, omega_perglobts_hat_glob_smth

ws_perglobts_smth = ccf_changes_perglobts(ts_amip,ws_amip, ts_hist,ws_hist,\
                                          ts_fast,ws_fast, ts_slow,ws_slow,smth=True) #, ws_perglobts_glob_smth, ws_perglobts_hat_smth, ws_perglobts_hat_zonal_smth, ws_perglobts_hat_glob_smth

In [ ]:
# dCCF_dTg_smth = [sst_perglobts_smth,
#             eis_perglobts_smth,
#             tadv_perglobts_smth,
#             rh_perglobts_smth,
#             omega_perglobts_smth,
#             ws_perglobts_smth]

# with open('data_smth/dCCF_dTg_smoothed.pkl', 'wb') as fp:
#     pickle.dump(dCCF_dTg_smth, fp)
#     print('dictionary saved successfully to file')

##### dR_cloud/dT
- `dR_ccf`,`dR_ccf_[obs]`: [lat,lon,exp] 16var - per model 
- `dR_ccf_perglobts_hat`,`dR_ccf_perglobts_hat_[obs]`: [lat,lon,exp], 1 var = (Multi-Model-Mean)MMM-ccf * kernel

In [ ]:
#  Radiative Feedbacks ccf-permodel*kernel-permod AND  ccf-permod * obs-kernel|MM-kernel 
dR_sst_perglobts,dR_sst_perglobts_ceres,dR_sst_perglobts_isccp,dR_sst_perglobts_modis,dR_sst_perglobts_patmosx,dR_sst_perglobts_mmkern = get_feedback(sst_perglobts.to_dataset(dim='model'),sst_kernels,obs_kernels=sst_obs_kernels)
dR_eis_perglobts,dR_eis_perglobts_ceres,dR_eis_perglobts_isccp,dR_eis_perglobts_modis,dR_eis_perglobts_patmosx,dR_eis_perglobts_mmkern = get_feedback(eis_perglobts.to_dataset(dim='model'),eis_kernels,obs_kernels=eis_obs_kernels)
dR_tadv_perglobts,dR_tadv_perglobts_ceres,dR_tadv_perglobts_isccp,dR_tadv_perglobts_modis,dR_tadv_perglobts_patmosx,dR_tadv_perglobts_mmkern = get_feedback(tadv_perglobts.to_dataset(dim='model'),tadv_kernels,obs_kernels=tadv_obs_kernels)
dR_rh_perglobts,dR_rh_perglobts_ceres,dR_rh_perglobts_isccp,dR_rh_perglobts_modis,dR_rh_perglobts_patmosx,dR_rh_perglobts_mmkern = get_feedback(rh_perglobts.to_dataset(dim='model'),rh_kernels,obs_kernels=rh_obs_kernels)
dR_omega_perglobts,dR_omega_perglobts_ceres,dR_omega_perglobts_isccp,dR_omega_perglobts_modis,dR_omega_perglobts_patmosx,dR_omega_perglobts_mmkern = get_feedback(omega_perglobts.to_dataset(dim='model'),omega_kernels,obs_kernels=omega_obs_kernels)
dR_ws_perglobts,dR_ws_perglobts_ceres,dR_ws_perglobts_isccp,dR_ws_perglobts_modis,dR_ws_perglobts_patmosx,dR_ws_perglobts_mmkern = get_feedback(ws_perglobts.to_dataset(dim='model'),ws_kernels,obs_kernels=ws_obs_kernels)

In [ ]:
#  Radiative Feedbacks [model, lat,lon,exp] MMM-ccf*kernel-permod - showing model kernel spread  
dR_sst_perglobts_mmccf = get_feedback(ensemble_mean(sst_perglobts),sst_kernels).to_array(dim='model')
dR_eis_perglobts_mmccf = get_feedback(ensemble_mean(eis_perglobts),eis_kernels).to_array(dim='model')
dR_tadv_perglobts_mmccf = get_feedback(ensemble_mean(tadv_perglobts),tadv_kernels).to_array(dim='model')
dR_rh_perglobts_mmccf= get_feedback(ensemble_mean(rh_perglobts),rh_kernels).to_array(dim='model')
dR_omega_perglobts_mmccf = get_feedback(ensemble_mean(omega_perglobts),omega_kernels).to_array(dim='model')
dR_ws_perglobts_mmccf = get_feedback(ensemble_mean(ws_perglobts),ws_kernels).to_array(dim='model')

In [ ]:
# dR [model,exp,lat,lon] - with 6+1 ccfs (from ccf-permod * kernel-permod) 
dR_modelccf_modelkern = ccf_feedbacks(dR_sst_perglobts,dR_eis_perglobts,dR_tadv_perglobts,dR_rh_perglobts,dR_omega_perglobts,dR_ws_perglobts)
dR_modelccf_cereskern = ccf_feedbacks(dR_sst_perglobts_ceres,dR_eis_perglobts_ceres,dR_tadv_perglobts_ceres,dR_rh_perglobts_ceres,dR_omega_perglobts_ceres,dR_ws_perglobts_ceres)
dR_modelccf_isccpkern = ccf_feedbacks(dR_sst_perglobts_isccp,dR_eis_perglobts_isccp,dR_tadv_perglobts_isccp,dR_rh_perglobts_isccp,dR_omega_perglobts_isccp,dR_ws_perglobts_isccp)
dR_modelccf_modiskern = ccf_feedbacks(dR_sst_perglobts_modis,dR_eis_perglobts_modis,dR_tadv_perglobts_modis,dR_rh_perglobts_modis,dR_omega_perglobts_modis,dR_ws_perglobts_modis)
dR_modelccf_patmoskern = ccf_feedbacks(dR_sst_perglobts_patmosx,dR_eis_perglobts_patmosx,dR_tadv_perglobts_patmosx,dR_rh_perglobts_patmosx,dR_omega_perglobts_patmosx,dR_ws_perglobts_patmosx)

dR_modelccf_mmkern = ccf_feedbacks(dR_sst_perglobts_mmkern,dR_eis_perglobts_mmkern,dR_tadv_perglobts_mmkern,dR_rh_perglobts_mmkern,dR_omega_perglobts_mmkern,dR_ws_perglobts_mmkern)
dR_mmccf_modelkern = ccf_feedbacks(dR_sst_perglobts_mmccf,dR_eis_perglobts_mmccf,dR_tadv_perglobts_mmccf,dR_rh_perglobts_mmccf,dR_omega_perglobts_mmccf,dR_ws_perglobts_mmccf)

# glob values for fig1 [model,exp] - with 6+1 ccf+tot
dR_glob_modelccf_modelkern = spatial_weighted_mean(dR_modelccf_modelkern)
dR_glob_modelccf_cereskern = spatial_weighted_mean(dR_modelccf_cereskern)
dR_glob_modelccf_isccpkern = spatial_weighted_mean(dR_modelccf_isccpkern)
dR_glob_modelccf_modiskern = spatial_weighted_mean(dR_modelccf_modiskern)
dR_glob_modelccf_patmoskern = spatial_weighted_mean(dR_modelccf_patmoskern)
dR_glob_modelccf_mmkern = spatial_weighted_mean(dR_modelccf_mmkern)
dR_glob_mmccf_modelkern = spatial_weighted_mean(dR_mmccf_modelkern)

## MMM of feedback (from ccf-permod * kernel-permod) (Spatial Maps)
dR_modelccf_modelkern_hat, dR_modelccf_modelkern_hat_zonal = hat_and_zonal(dR_modelccf_modelkern)

In [ ]:
# df = dR_glob_modelccf_modelkern.to_dataframe().reset_index()

# df_ceres = dR_glob_modelccf_cereskern.to_dataframe().reset_index()
# df_isccp = dR_glob_modelccf_isccpkern.to_dataframe().reset_index()
# df_modis = dR_glob_modelccf_modiskern.to_dataframe().reset_index()
# df_patmos = dR_glob_modelccf_patmoskern.to_dataframe().reset_index()

# df_mmkern = dR_glob_modelccf_mmkern.to_dataframe().reset_index()
# df_mmccf = dR_glob_mmccf_modelkern.to_dataframe().reset_index() 

# df.to_csv('df.csv')
# df_ceres.to_csv('df_ceres.csv')
# df_isccp.to_csv('df_isccp.csv')
# df_modis.to_csv('df_modis.csv')
# df_patmos.to_csv('df_patmosx.csv')
# df_mmccf.to_csv('df_mmccf.csv')
# df_mmkern.to_csv('df_mmkern.csv')

##### Var(dR_cloud/dT)

In [ ]:
# Variance for each dR_ccf (for supp plots)
var_dR_sst = indv_var_calc(sst_kernels,sst_perglobts,dR_modelccf_modelkern,"sst")
var_dR_eis = indv_var_calc(eis_kernels,eis_perglobts,dR_modelccf_modelkern,"eis")
var_dR_tadv = indv_var_calc(tadv_kernels,tadv_perglobts,dR_modelccf_modelkern,"tadv")
var_dR_rh = indv_var_calc(rh_kernels,rh_perglobts,dR_modelccf_modelkern,"rh")
var_dR_omega = indv_var_calc(omega_kernels,omega_perglobts,dR_modelccf_modelkern,"omega")
var_dR_ws = indv_var_calc(ws_kernels,ws_perglobts,dR_modelccf_modelkern,"ws")

In [ ]:
## Calculate for tot  
dR_sst_kp_cp,dR_sst_km_cp,dR_sst_kp_cm,dR_sst_km_cm = prime_mmm_calc(sst_kernels,sst_perglobts)
dR_eis_kp_cp,dR_eis_km_cp,dR_eis_kp_cm,dR_eis_km_cm = prime_mmm_calc(eis_kernels,eis_perglobts)
dR_tadv_kp_cp,dR_tadv_km_cp,dR_tadv_kp_cm,dR_tadv_km_cm = prime_mmm_calc(tadv_kernels,tadv_perglobts)
dR_rh_kp_cp,dR_rh_km_cp,dR_rh_kp_cm,dR_rh_km_cm = prime_mmm_calc(rh_kernels,rh_perglobts)
dR_omega_kp_cp,dR_omega_km_cp,dR_omega_kp_cm,dR_omega_km_cm = prime_mmm_calc(omega_kernels,omega_perglobts)
dR_ws_kp_cp,dR_ws_km_cp,dR_ws_kp_cm,dR_ws_km_cm = prime_mmm_calc(ws_kernels,ws_perglobts)

dR_modk_modc_var = (dR_modelccf_modelkern['tot'].var('model')).compute() #col1

tot_kp_cm = dR_sst_kp_cm+dR_eis_kp_cm+dR_tadv_kp_cm+dR_rh_kp_cm+dR_omega_kp_cm+dR_ws_kp_cm
tot_kp_cm_var = tot_kp_cm.var(dim='model').compute() #col2
tot_km_cp = dR_sst_km_cp+dR_eis_km_cp+dR_tadv_km_cp+dR_rh_km_cp+dR_omega_km_cp+dR_ws_km_cp
tot_km_cp_var = tot_km_cp.var(dim='model').compute() #col3

tot_aprx_var =  (tot_km_cp+tot_kp_cm).var(dim='model').compute()
var_diff = (dR_modk_modc_var - tot_aprx_var).compute() #col4 

In [ ]:
# dR_ccf_var = [var_dR_sst,var_dR_eis,var_dR_tadv,var_dR_rh,var_dR_omega,var_dR_ws]
# with open('dR_ccf_var.pkl', 'wb') as fp:
#     pickle.dump(dR_ccf_var, fp)
#     print('dictionary saved successfully to file')

# dR_tot_var = [dR_modk_modc_var,tot_kp_cm_var,tot_km_cp_var,var_diff]
# with open('dR_tot_var.pkl', 'wb') as fp:
#     pickle.dump(dR_tot_var, fp)
#     print('dictionary saved successfully to file')